In [ ]:
import numpy as np
import scipy.io as sio
import pyamg
import scipy.sparse.linalg as sla

import matplotlib.pyplot as plt
from matplotlib import collections
from matplotlib import tri
from common import set_figure

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Read in a problem

This is a poisson problem on an unstructured mesh

`A` is the matrix

`vertices` are the vertices in the mesh

`elements` are the triangles in the mesh

In [ ]:
data = pyamg.gallery.load_example('unit_square')

A = data['A'].tocsr()                        # matrix
V = data['vertices'][:A.shape[0]]            # vertices of each variable
Elmts = data['elements']
n = A.shape[0]

x = V[:,0]
y = V[:,1]

# Plot the mesh

In [ ]:
f, ax = plt.subplots(1, figsize=(8,8))
t = tri.Triangulation(x, y, Elmts)
plt.triplot(t)
plt.axis('off')

# Plot the "graph of $A$"

In [ ]:
thismatrix = A
E = np.vstack((thismatrix.tocoo().row,thismatrix.tocoo().col)).T  # edges of the matrix graph
print(E)
f, ax = plt.subplots(1, figsize=(8,8))

lines = np.empty((E.shape[0], 2, 2))
lines[:,0,0] = x[E[:,0]] # xstart
lines[:,1,0] = x[E[:,1]] # xend
lines[:,0,1] = y[E[:,0]] # ystart
lines[:,1,1] = y[E[:,1]] # yend

ls = collections.LineCollection(lines)
ax.add_collection(ls, autolim=True)
ax.autoscale_view()
ax.axis('off')

# Set the near null space or candidate vectors

In [ ]:
B = np.ones((n,1))

# Find the strength of connection in the graph of $A$

In [ ]:
S = pyamg.strength.symmetric_strength_of_connection(A, theta = 0.1)
S

In [ ]:
def plotmatrix(thismatrix, ax, lw=1, color='tab:blue'):
    E = np.vstack((thismatrix.tocoo().row,thismatrix.tocoo().col)).T  # edges of the matrix graph

    lines = np.empty((E.shape[0], 2, 2))
    lines[:,0,0] = x[E[:,0]] # xstart
    lines[:,1,0] = x[E[:,1]] # xend
    lines[:,0,1] = y[E[:,0]] # ystart
    lines[:,1,1] = y[E[:,1]] # yend

    ls = collections.LineCollection(lines, color=color)
    ax.add_collection(ls, autolim=True)
    ls.set_linewidth(lw)
    ax.autoscale_view()
    ax.axis('off')

In [ ]:
f, ax = plt.subplots(1, figsize=(6,6))

# FIRST A
plotmatrix(A, ax, color='tab:blue')

# THEN S
plotmatrix(S, ax, lw=6, color='tab:red')

# What about point 44?
i = 44
J = A.getrow(i).indices
for j in J:
    plt.plot(x[j], y[j], 'ro')
    plt.text(x[j], y[j], '%d'%j, fontsize=12, color='k')
    
print(A.getrow(i))

# Aggregate based on Strength

In [ ]:
AggOp, Cpts = pyamg.aggregation.aggregate.standard_aggregation(S)

In [ ]:
AggOp

In [ ]:
f, ax = plt.subplots(1, figsize=(6,6))

plotmatrix(A, ax)

plotmatrix(S, ax, lw=2)

# now plot each aggregate
for i in range(AggOp.shape[1]):
    J = AggOp.getcol(i).tocoo().row
    for j1 in J:
        for j2 in J:    
            if j1 != j2:
                if A[j1, j2]:
                    plt.plot([x[j1], x[j2]], [y[j1], y[j2]], 'r', lw=4)
    if len(J) == 1:
        plt.plot(x[J[0]], y[J[0]], 'ro', ms=10)
                    
for i, v in enumerate(V):
    plt.text(v[0], v[1], '%d'%i, fontsize=10)

# Create an interpolation operator

In [ ]:
plt.spy(AggOp, ms=2)

In [ ]:
T, R = pyamg.aggregation.tentative.fit_candidates(AggOp, B)

In [ ]:
T
row = T.getrow(93)
print(row.indices)

# Improve the interpolation operator

In [ ]:
P = pyamg.aggregation.smooth.jacobi_prolongation_smoother(A, T, S, B, degree=6).tocsr()

In [ ]:
T = T.toarray()
P = P.toarray()

In [ ]:
f, ax = plt.subplots(1, figsize=(4,4))
t = tri.Triangulation(x, y, Elmts)
plt.tripcolor(t, T[:,18])
plt.axis('off')

f, ax = plt.subplots(1, figsize=(4,4))
t = tri.Triangulation(x, y, Elmts)
plt.tripcolor(t, P[:,18])
plt.axis('off')

## A full multilevel hierarchy

In [ ]:
ml = pyamg.smoothed_aggregation_solver(A, 
                                       strength=('evolution', {'epsilon': 8.0}),
                                       max_coarse=1, keep=True, improve_candidates=None)
print(ml)

In [ ]:
res = []
b = np.zeros((n,))
x = np.random.rand(n)
x = ml.solve(b, x0=x, residuals=res)
print(res)

In [ ]:
res = np.array(res)
res[1:] / res[:-1]